## CNN Model

In [1]:
# importing function 
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, MaxPooling1D, Bidirectional,LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D, Flatten , Embedding, GlobalMaxPool1D
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv2D, MaxPooling2D


In [3]:
# training data 
cnn_df = pd.read_csv("data_with_features.csv", dtype='unicode')
cnn_df = cnn_df.drop(['Return-Path','Message-ID','From','Reply-To','To','Submitting Host','Subject','Date','X-Mailer','MIME-Version','Content-Type','X-Priority','X-MSMail-Priority','Status','Content-Length','Content-Transfer-Encoding','Lines','hops'], axis = 1)
cnn_df.head()

,Label,special_characters_exists_subject,number_of_words_subject,number_of_capitalized_words_subject,number_of_capitalized_characters_subject,number_of_digits_subject,number_of_characters_subject,number_of_spaces_subject,number_of_special_characters_subject,number_of_single_Quotes_subject,number_of_semiColon_subject,ratio_of_uppercase/lowercase_words,Total_number_of_upperCase,Max_word_length_in_subject
0,1,0,4,0,0,0,19,4,0,0.0,0,0.0,0,9
1,0,0,3,0,0,0,9,2,0,0.0,0,0.0,0,6
2,0,1,6,3,7,0,15,6,4,0.0,0,1.0,3,8
3,1,0,3,0,2,0,12,2,0,0.0,0,0.0,0,7
4,0,0,2,0,1,0,2,1,0,0.0,0,0.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62389,0,0,2,0,1,0,2,1,0,0.0,0,0.0,0,2
62390,0,1,4,0,1,7,7,3,1,0.0,0,0.0,0,6
62391,1,0,4,1,7,0,26,4,0,0.0,0,0.3333333333333333,1,11
62392,0,0,2,0,1,0,2,1,0,0.0,0,0.0,0,2


#### Split training data into labels and features

In [4]:
test_size = int(len(cnn_df) * 0.5)
train_data = cnn_df.iloc[:-test_size,:].copy()
test_data = cnn_df.iloc[-test_size:,:].copy()

In [5]:
features_train = train_data.drop('Label',axis=1).copy()
label_train = train_data[['Label']].copy()

#### Merge feature training and label training

In [6]:
feature_scaler = MinMaxScaler(feature_range=(0, 1))
feature_scaler.fit(features_train)
scaled_feature_train = feature_scaler.transform(features_train)

label_scaler = MinMaxScaler(feature_range=(0, 1))
label_scaler.fit(label_train)
scaled_label_train = label_scaler.transform(label_train)

scaled_label_train = scaled_label_train.reshape(-1)


scaled_label_train = np.insert(scaled_label_train, 0, 0)
scaled_label_train = np.delete(scaled_label_train, -1)

In [7]:
n_input = 25 
n_features= features_train.shape[1]
b_size = 32
generator = TimeseriesGenerator(scaled_feature_train, scaled_label_train, length=n_input, batch_size=b_size)

print(generator[0][0].shape)

(32, 25, 13)


#### build the sequential model

In [18]:
model = Sequential()

model.add(Conv1D(128, input_shape=(n_input, n_features), kernel_size=5))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Conv1D(128,kernel_size=5))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 21, 128)           8448      
_________________________________________________________________
activation_7 (Activation)    (None, 21, 128)           0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 21, 128)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 17, 128)           82048     
_________________________________________________________________
activation_8 (Activation)    (None, 17, 128)           0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 17, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2176)            

In [22]:
model.fit(generator,epochs=11)

Epoch 1/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1802 - accuracy: 0.9289
Epoch 2/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1697 - accuracy: 0.9328
Epoch 3/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1560 - accuracy: 0.9382
Epoch 4/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1473 - accuracy: 0.9418
Epoch 5/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1352 - accuracy: 0.9475
Epoch 6/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1302 - accuracy: 0.9488
Epoch 7/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1191 - accuracy: 0.9542
Epoch 8/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1114 - accuracy: 0.9566
Epoch 9/11
975/975 [==============================] - 4s 4ms/step - loss: 0.1042 - accuracy: 0.9581
Epoch 10/11
975/975 [==============================] - 4s 4ms/step - loss: 0.0977 - accuracy: 0.9615